In [1]:
import glob
import json
import os
import re
import time
import wget
import urllib.parse
import argparse


import numpy as np
import pandas as pd
import pubchempy as pcp


from pybatchclassyfire import *
from pandas import json_normalize
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import PandasTools
def isNaN(string):
    return string != string

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [2]:
path = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-Diatom/SmarinoiRun1"
file = os.listdir(path)
folders = [x for x in os.listdir(path) if x.startswith('DS')]
folders2 = [x for x in folders if not '.mzML' in x]
folders2

['DS_201124_SC_full_PRM_pos_03',
 'DS_201124_SC_full_PRM_pos_04',
 'DS200309_Scost_QC_70k_neg_PRM',
 'DS_201124_SC_full_PRM_pos_05',
 'DS_201124_SC_full_PRM_pos_02',
 'DS_201124_SC_full_PRM_neg_03',
 'DS_201124_SC_full_PRM_neg_04',
 'DS200309_Scost_QC_70k_pos_PRM',
 'DS_201124_SC_full_PRM_neg_05',
 'DS_201124_SC_full_PRM_neg_02',
 'DS_201124_SC_full_PRM_neg_10',
 'DS_201124_SC_full_PRM_pos_10',
 'DS_201124_SC_full_PRM_pos_07',
 'DS_201124_SC_full_PRM_pos_09',
 'DS_201124_SC_full_PRM_pos_08',
 'DS_201124_SC_full_PRM_pos_01',
 'DS_201124_SC_full_PRM_pos_06',
 'DS_201124_SC_full_PRM_neg_07',
 'DS_201124_SC_full_PRM_neg_09',
 'DS_201124_SC_full_PRM_neg_08',
 'DS_201124_SC_full_PRM_neg_01',
 'DS_201124_SC_full_PRM_neg_06']

In [3]:
for i in folders2:
    entry = path + "/" + i
    sub_dir = entry + "/insilico/SIRIUS/"
    file = os.listdir(sub_dir)
    foldersir = [x for x in os.listdir(sub_dir) if x.endswith('.json')]
    foldersir = [x for x in foldersir if 'SList.json' in x]
    print(entry)
    sirius_dirs = foldersir
    sirius_postproc(entry, sirius_dirs, db = "SL")

/Users/mahnoorzulfiqar/OneDriveUNI/MAW-Diatom/SmarinoiRun1/DS_201124_SC_full_PRM_pos_03


NameError: name 'sirius_postproc' is not defined

In [167]:
entry

'/Users/mahnoorzulfiqar/OneDriveUNI/MAW-Diatom/SmarinoiRun1/DS_201124_SC_full_PRM_pos_09'

In [172]:
if os.path.isdir(entry + "/insilico/SIRIUS/"):
    sub_dir = entry + "/insilico/SIRIUS/"
    msp_csv = entry + "/insilico/MS1DATA.csv"
    if os.path.exists(msp_csv) and os.path.exists(sub_dir):
        # output json files from SIRIUS
        files_S = sirius_dirs
        # list of precursor m/z
        msp = pd.read_csv(msp_csv)
        # for each mz
        for mz, row in msp.iterrows():
            # make a list of files with this mz
            files_for_mz = []

            for file in files_S:
                if str(msp["premz"][mz]) in file:
                    files_for_mz.append(file)

#             # extract the formula and structure files
#             json_dirALL = next(os.walk(sub_dir + files_for_mz[0]))[1]
            

In [173]:
files_for_mz

[]

In [ ]:

        
            if len(json_dirALL) == 1:
                sub_sub_dirALL_structure_can = (sub_dir +
                    files_for_mz[0]
                    + "/"
                    + json_dirALL[0]
                    + "/structure_candidates.tsv"
                )
                sub_sub_dirALL_formula_can = (sub_dir +
                    files_for_mz[0]
                    + "/"
                    + json_dirALL[0]
                    + "/formula_candidates.tsv"
                )
                ALL_Canopus_csv = sub_dir + files_for_mz[0] + "/canopus_summary.tsv"

                # if both structure files exist
                if (
                    os.path.exists(sub_sub_dirALL_structure_can)
                    and len(pd.read_csv(sub_sub_dirALL_structure_can, sep="\t")) > 0
                ):
                    if (
                         os.path.exists(sub_sub_dirALL_formula_can)
                        and len(pd.read_csv(sub_sub_dirALL_formula_can, sep="\t"))
                        > 0
                    ):
                        ALL_structure_csv = pd.read_csv(
                            sub_sub_dirALL_structure_can, sep="\t"
                        )
                        ALL_formula_csv = pd.read_csv(
                            sub_sub_dirALL_formula_can, sep="\t"
                        )
                        ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep="\t")
                        # Add the structure and formula files together
                        for structure, rows in ALL_structure_csv.iterrows():
                            for formula, rows in ALL_formula_csv.iterrows():
                                if (
                                    ALL_structure_csv["formulaRank"][structure]
                                    == ALL_formula_csv["rank"][formula]
                                ):
                                    ALL_structure_csv.loc[
                                        structure, "SiriusScore"
                                    ] = ALL_formula_csv["SiriusScore"][formula]
                                    ALL_structure_csv.loc[
                                        structure, "numExplainedPeaks"
                                    ] = ALL_formula_csv["numExplainedPeaks"][
                                        formula
                                    ]
                                    ALL_structure_csv.loc[
                                        structure, "explainedIntensity"
                                    ] = ALL_formula_csv["explainedIntensity"][
                                        formula
                                    ]
                                    # ALL_structure_csv.loc[structure, "SuspectListEntry"] = "FALSE"
                                    if len(ALL_Canopus) > 0:
                                        if (
                                            ALL_formula_csv["molecularFormula"][
                                                formula
                                            ]
                                            == ALL_Canopus["molecularFormula"][0]
                                        ):
                                            ALL_structure_csv.loc[
                                                structure, "superclass"
                                            ] = ALL_Canopus["superclass"][0]
                                            ALL_structure_csv.loc[
                                                structure, "class"
                                            ] = ALL_Canopus["class"][0]
                                            ALL_structure_csv.loc[
                                                structure, "subclass"
                                            ] = ALL_Canopus["subclass"][0]

#                                 for str_siriusA, row in ALL_structure_csv.iterrows():
#                                     if not str_can_score(ALL_structure_csv, str_siriusA):
#                                         ALL_structure_csv = ALL_structure_csv.drop(
#                                             str_siriusA, inplace=False
#                                         )

                    result_sirius_name = (
                        sub_dir
                        + db
                        + "_results_for_"
                        + json_dirALL[0].split("_")[-1]
                        + "_"
                        + "structure.csv"
                    )
                    msp.loc[mz, "sirius_result_dir"] = result_sirius_name

                    ALL_structure_csv.to_csv(result_sirius_name)

                elif not (
                    os.path.exists(sub_sub_dirALL_structure_can)
                    and len(pd.read_csv(sub_sub_dirALL_structure_can, sep="\t")) == 0
                ):
                    if (
                        os.path.exists(sub_sub_dirALL_formula_can)
                        and len(pd.read_csv(sub_sub_dirALL_formula_can, sep="\t"))
                        > 0
                    ):
                        ALL_formula_csv = pd.read_csv(
                            sub_sub_dirALL_formula_can, sep="\t"
                        )
                        ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep="\t")
                        if len(ALL_Canopus) > 0:
                            for formula, rows in ALL_formula_csv.iterrows():
                                ALL_formula_csv.loc[
                                    formula, "superclass"
                                ] = ALL_Canopus["superclass"][0]
                                ALL_formula_csv.loc[formula, "class"] = ALL_Canopus[
                                    "class"
                                ][0]
                                ALL_formula_csv.loc[
                                    formula, "subclass"
                                ] = ALL_Canopus["subclass"][0]

                        for for_siriusA, row in ALL_formula_csv.iterrows():
                            if (
                                not ALL_formula_csv["explainedIntensity"][
                                    for_siriusA
                                ]
                                >= exp_int
                            ):
                                ALL_formula_csv = ALL_formula_csv.drop(
                                    for_siriusA, inplace=False
                                )

                        result_sirius_name = (
                            sub_dir
                            + db
                            + "_results_for_"
                            + json_dirALL[0].split("_")[-1]
                            + "_"
                            + "formula.csv"
                        )
                        msp.loc[
                            mz, "sirius_result_dir"
                        ] = result_sirius_name

                        ALL_formula_csv.to_csv(
                            sub_dir
                            + db
                            + "_results_for_"
                            + json_dirALL[0].split("_")[-1]
                            + "_"
                            + "formula.csv"
                        )

                    else:
                        print("no file for formula")
                else:
                    print("no file for structure or formula")
        msp.to_csv(msp_csv)

In [179]:
folders2

['DS_201124_SC_full_PRM_pos_03',
 'DS_201124_SC_full_PRM_pos_04',
 'DS200309_Scost_QC_70k_neg_PRM',
 'DS_201124_SC_full_PRM_pos_05',
 'DS_201124_SC_full_PRM_pos_02',
 'DS_201124_SC_full_PRM_neg_03',
 'DS_201124_SC_full_PRM_neg_04',
 'DS200309_Scost_QC_70k_pos_PRM',
 'DS_201124_SC_full_PRM_neg_05',
 'DS_201124_SC_full_PRM_neg_02',
 'DS_201124_SC_full_PRM_neg_10',
 'DS_201124_SC_full_PRM_pos_10',
 'DS_201124_SC_full_PRM_pos_07',
 'DS_201124_SC_full_PRM_pos_09',
 'DS_201124_SC_full_PRM_pos_08',
 'DS_201124_SC_full_PRM_pos_01',
 'DS_201124_SC_full_PRM_pos_06',
 'DS_201124_SC_full_PRM_neg_07',
 'DS_201124_SC_full_PRM_neg_09',
 'DS_201124_SC_full_PRM_neg_08',
 'DS_201124_SC_full_PRM_neg_01',
 'DS_201124_SC_full_PRM_neg_06']

In [177]:
for i in folders2:
    entry = path + "/" + i
    sub_dir = entry + "/insilico/SIRIUS/"
    file = os.listdir(sub_dir)
    foldersir = [x for x in os.listdir(sub_dir) if x.endswith('.json')]
    foldersir = [x for x in foldersir if not 'SList.json' in x]
    #print(foldersir)
    sirius_dirs = foldersir
    sirius_postproc(entry, sirius_dirs, db = "ALL")

['34_NA_iso_NA_MS1p_563.550964355469_SIRIUS_param.json', '63_NA_iso_NA_MS1p_104.106819152832_SIRIUS_param.json', '8_NA_iso_NA_MS1p_157.974563598633_SIRIUS_param.json', '5_NA_iso_NA_MS1p_159.969161987305_SIRIUS_param.json', '59_NA_iso_MS1p_182.081237792969_SIRIUS_param.json', '37_NA_iso_NA_MS1p_511.288116455078_SIRIUS_param.json', '74_NA_iso_NA_MS1p_99.531120300293_SIRIUS_param.json', '52_NA_iso_NA_MS1p_120.080688476563_SIRIUS_param.json', '69_NA_iso_MS1p_290.207183837891_SIRIUS_param.json', '28_NA_iso_MS1p_249.148559570313_SIRIUS_param.json', '26_NA_iso_NA_MS1p_642.2685546875_SIRIUS_param.json', '12_NA_iso_NA_MS1p_314.935882568359_SIRIUS_param.json', '55_NA_iso_NA_MS1p_112.050437927246_SIRIUS_param.json', '56_NA_iso_NA_MS1p_162.075805664063_SIRIUS_param.json', '2_NA_iso_NA_MS1p_137.964111328125_SIRIUS_param.json', '72_NA_iso_NA_MS1p_82.013771057129_SIRIUS_param.json', '18_NA_iso_NA_MS1p_180.989562988281_SIRIUS_param.json', '21_NA_iso_NA_MS1p_357.150909423828_SIRIUS_param.json', '20_iso

no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
['78_iso_MS1p_387.112487792969_SIRIUS_param.json', '133_NA_iso_NA_MS1p_302.100494384766_SIRIUS_param.json', '103_iso_MS1p_333.041351318359_SIRIUS_param.json', '99_NA_iso_NA_MS1p_267.070861816406_SIRIUS_param.json', '37_NA_iso_NA_MS1p_291.064849853516_SIRIUS_param.json', '112_NA_iso_MS1p_203.103225708008_SIRIUS_param.json', '127_NA_iso_NA_MS1p_451.120178222656_SIRIUS_param.json', '18_NA_iso_NA_MS1p_191.055053710938_SIRIUS_param.json', '79_NA_iso_NA_MS1p_395.134765625_SIRIUS_param.json', '66_NA_iso_NA_MS1p_496.9837

no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
['7_NA_iso_NA_MS1p_117.959426879883_SIRIUS_param.json', '1_NA_iso_NA_MS1p_89.506637573242_SIRIUS_param.json', '72_NA_iso_NA_MS1p_83.037406921387_SIRIUS_param.json', '11_NA_iso_NA_MS1p_181.988204956055_SIRIUS_param.json', '45_NA_iso_NA_MS1p_232.190628051758_SIRIUS_param.json', '27_NA_iso_MS1p_181.122268676758_SIRIUS_param.json', '57_NA_iso_NA_MS1p_112.999671936035_SIRIUS_param.json', '50_NA_iso_MS1p_244.263397216797_SIRIUS_param.json', '34_NA_iso_NA_MS1p_607.291564941406_SIRIUS_param.json', '43_NA_iso_NA_MS1p_275.073669433594_SIRIUS_param.json', '16_NA_iso_NA_MS1p_369.351318359375_SIRIUS_param.json', '64_NA_iso_MS1p_531.123352050781_SIRIUS_param.json', '29_iso_MS1p_325.200866699219_SIRIUS_param.json', '28_NA_iso_MS1p_256.227325439453_SIRIUS_param.json', '38_iso_MS1p_323.221343994141_SIRIUS_param.json', '18_NA_iso_NA_MS1p_595

no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
['5_NA_iso_NA_MS1p_190.928466796875_SIRIUS_param.json', '55_iso_MS1p_290.073028564453_SIRIUS_param.json', '9_NA_iso_NA_MS1p_698.503601074219_SIRIUS_param.json', '15_iso_MS1p_278.93017578125_SIRIUS_param.json', '62_NA_iso_MS1p_154.017868041992_SIRIUS_param.json', '10_iso_MS1p_763.467346191406_SIRIUS_param.json', '36_NA_iso_NA_MS1p_265.050048828125_SIRIUS_param.json', '63_iso_MS1p_199.025756835938_SIRIUS_param.json', '13_NA_iso_NA_MS1p_467.241485595703_SIRIUS_param.json', '37_NA_iso_MS1p_240.018524169922_SIRIUS_param.json', '19_NA_iso_NA_MS1p_322.865234375_SIRIUS_param.json', '20_NA_iso_MS1p_283.155242919922_SIR

no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for stru

...

no file for formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
['24_NA_iso_NA_MS1p_365.188140869141_SIRIUS_param.json', '8_NA_iso_NA_MS1p_223.988174438477_SIRIUS_param.json', '48_NA_iso_MS1p_520.339233398438_SIRIUS_param.json', '4_NA_iso_NA_MS1p_142.990921020508_SIRIUS_param.json', '7_NA_iso_NA_MS1p_145.980697631836_SIRIUS_param.json', '11_NA_iso_NA_MS1p_197.000793457031_SIRIUS_param.json', '57_NA_iso_NA_MS1p_176.992156982422_SIRIUS_param.json', '32_NA_iso_MS1p_226.180130004883_SIRIUS_param.json', '13_NA_iso_NA_MS1p_197.000778198242_SIRIUS_param.json', '36_NA_iso_MS1p_237.221115112305_SIRIUS_param.json', '63_NA_iso_MS1p_146.117431640625_SIRIUS_p

no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
['6_NA_iso_NA_MS1p_328.915130615234_SIRIUS_param.json', '33_NA_iso_MS1p_305.210876464844_SIRIUS_param.json', '23_iso_MS1p_249.185119628906_SIRIUS_param.json', '50_NA_iso_NA_MS1p_272.294677734375_SIRIUS_param.json', '2_NA_iso_NA_MS1p_182.985122680664_SIRIUS_param.json', '31_NA_iso_NA_MS1p_739.438903808594_SIRIUS_param.json', '46_NA_iso_NA_MS1p_404.759582519531_SIRIUS_param.json', '11_NA_iso_NA_MS1p_183.009902954102_SIRIUS_param.json', '64_iso_MS1p_302.114074707031_SIRIUS_param.json', '60_NA_iso_MS1p_256.128997802734_SIRIUS_param.json', '57_NA_iso_NA_MS1p_187.036422729492_SIRIUS_param.json', '15_NA_iso_NA_MS1p_351.886627197266_SIRI

no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for stru

no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for stru

IndexError: list index out of range

In [191]:
entry

'/Users/mahnoorzulfiqar/OneDriveUNI/MAW-Diatom/SmarinoiRun1/DS_201124_SC_full_PRM_neg_06'

In [193]:
sub_dir = entry + "/insilico/SIRIUS/"
file = os.listdir(sub_dir)
foldersir = [x for x in os.listdir(sub_dir) if x.endswith('.json')]
foldersir = [x for x in foldersir if not 'SList.json' in x]
#print(foldersir)
sirius_dirs = foldersir
print(sirius_dirs)

['38_NA_iso_NA_MS1p_201.906127929688_SIRIUS_param.json', '35_NA_iso_NA_MS1p_254.07063293457_SIRIUS_param.json', '50_NA_iso_NA_MS1p_160.935470581055_SIRIUS_param.json', '11_NA_iso_NA_MS1p_553.269226074219_SIRIUS_param.json', '46_NA_iso_MS1p_315.833618164063_SIRIUS_param.json', '45_NA_iso_NA_MS1p_278.769378662109_SIRIUS_param.json', '68_NA_iso_NA_MS1p_96.960182189941_SIRIUS_param.json', '31_NA_iso_NA_MS1p_174.964248657227_SIRIUS_param.json', '58_NA_iso_NA_MS1p_232.853866577148_SIRIUS_param.json', '7_NA_iso_MS1p_759.435607910156_SIRIUS_param.json', '47_NA_iso_NA_MS1p_248.741088867188_SIRIUS_param.json', '41_NA_iso_NA_MS1p_208.934646606445_SIRIUS_param.json', '28_NA_iso_NA_MS1p_364.889129638672_SIRIUS_param.json', '64_NA_iso_NA_MS1p_411.056427001953_SIRIUS_param.json', '2_NA_iso_MS1p_115.920883178711_SIRIUS_param.json', '25_NA_iso_NA_MS1p_316.779052734375_SIRIUS_param.json', '22_NA_iso_NA_MS1p_529.265625_SIRIUS_param.json', '37_NA_iso_NA_MS1p_564.330871582031_SIRIUS_param.json', '1_NA_iso_

In [194]:
sirius_postproc(entry, sirius_dirs, db = "ALL")

no file for formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or formula
no file for structure or form

In [190]:
def sirius_postproc(entry, sirius_dirs, db):
    if os.path.isdir(entry + "/insilico/SIRIUS/"):
        sub_dir = entry + "/insilico/SIRIUS/"
        msp_csv = entry + "/insilico/MS1DATA.csv"
        if os.path.exists(msp_csv) and os.path.exists(sub_dir):
            # output json files from SIRIUS
            files_S = sirius_dirs
            # list of precursor m/z
            msp = pd.read_csv(msp_csv)

            # for each mz
            for mz, row in msp.iterrows():
                # make a list of files with this mz
                files_for_mz = []

                for file in files_S:
                    if str(msp["premz"][mz]) in file:
                        files_for_mz.append(file)
                if len(files_for_mz) > 0:
                    # extract the formula and structure files
                    json_dirALL = next(os.walk(sub_dir + files_for_mz[0]))[1]

                    if len(json_dirALL) == 1:
                        sub_sub_dirALL_structure_can = (sub_dir +
                            files_for_mz[0]
                            + "/"
                            + json_dirALL[0]
                            + "/structure_candidates.tsv"
                        )
                        sub_sub_dirALL_formula_can = (sub_dir +
                            files_for_mz[0]
                            + "/"
                            + json_dirALL[0]
                            + "/formula_candidates.tsv"
                        )
                        ALL_Canopus_csv = sub_dir + files_for_mz[0] + "/canopus_summary.tsv"

                        # if both structure files exist
                        if (
                            os.path.exists(sub_sub_dirALL_structure_can)
                            and len(pd.read_csv(sub_sub_dirALL_structure_can, sep="\t")) > 0
                        ):
                            if (
                                 os.path.exists(sub_sub_dirALL_formula_can)
                                and len(pd.read_csv(sub_sub_dirALL_formula_can, sep="\t"))
                                > 0
                            ):
                                ALL_structure_csv = pd.read_csv(
                                    sub_sub_dirALL_structure_can, sep="\t"
                                )
                                ALL_formula_csv = pd.read_csv(
                                    sub_sub_dirALL_formula_can, sep="\t"
                                )
                                ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep="\t")
                                # Add the structure and formula files together
                                for structure, rows in ALL_structure_csv.iterrows():
                                    for formula, rows in ALL_formula_csv.iterrows():
                                        if (
                                            ALL_structure_csv["formulaRank"][structure]
                                            == ALL_formula_csv["rank"][formula]
                                        ):
                                            ALL_structure_csv.loc[
                                                structure, "SiriusScore"
                                            ] = ALL_formula_csv["SiriusScore"][formula]
                                            ALL_structure_csv.loc[
                                                structure, "numExplainedPeaks"
                                            ] = ALL_formula_csv["numExplainedPeaks"][
                                                formula
                                            ]
                                            ALL_structure_csv.loc[
                                                structure, "explainedIntensity"
                                            ] = ALL_formula_csv["explainedIntensity"][
                                                formula
                                            ]
                                            # ALL_structure_csv.loc[structure, "SuspectListEntry"] = "FALSE"
                                            if len(ALL_Canopus) > 0:
                                                if (
                                                    ALL_formula_csv["molecularFormula"][
                                                        formula
                                                    ]
                                                    == ALL_Canopus["molecularFormula"][0]
                                                ):
                                                    ALL_structure_csv.loc[
                                                        structure, "superclass"
                                                    ] = ALL_Canopus["superclass"][0]
                                                    ALL_structure_csv.loc[
                                                        structure, "class"
                                                    ] = ALL_Canopus["class"][0]
                                                    ALL_structure_csv.loc[
                                                        structure, "subclass"
                                                    ] = ALL_Canopus["subclass"][0]

    #                                 for str_siriusA, row in ALL_structure_csv.iterrows():
    #                                     if not str_can_score(ALL_structure_csv, str_siriusA):
    #                                         ALL_structure_csv = ALL_structure_csv.drop(
    #                                             str_siriusA, inplace=False
    #                                         )

                            result_sirius_name = (
                                sub_dir
                                + db
                                + "_results_for_"
                                + json_dirALL[0].split("_")[-1]
                                + "_"
                                + "structure.csv"
                            )
                            msp.loc[mz, "sirius_result_dir"] = result_sirius_name

                            ALL_structure_csv.to_csv(result_sirius_name)

                        elif not (
                            os.path.exists(sub_sub_dirALL_structure_can)
                            and len(pd.read_csv(sub_sub_dirALL_structure_can, sep="\t")) == 0
                        ):
                            if (
                                os.path.exists(sub_sub_dirALL_formula_can)
                                and len(pd.read_csv(sub_sub_dirALL_formula_can, sep="\t"))
                                > 0
                            ):
                                ALL_formula_csv = pd.read_csv(
                                    sub_sub_dirALL_formula_can, sep="\t"
                                )
                                ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep="\t")
                                if len(ALL_Canopus) > 0:
                                    for formula, rows in ALL_formula_csv.iterrows():
                                        ALL_formula_csv.loc[
                                            formula, "superclass"
                                        ] = ALL_Canopus["superclass"][0]
                                        ALL_formula_csv.loc[formula, "class"] = ALL_Canopus[
                                            "class"
                                        ][0]
                                        ALL_formula_csv.loc[
                                            formula, "subclass"
                                        ] = ALL_Canopus["subclass"][0]

                                for for_siriusA, row in ALL_formula_csv.iterrows():
                                    if (
                                        not ALL_formula_csv["explainedIntensity"][
                                            for_siriusA
                                        ]
                                        >= exp_int
                                    ):
                                        ALL_formula_csv = ALL_formula_csv.drop(
                                            for_siriusA, inplace=False
                                        )

                                result_sirius_name = (
                                    sub_dir
                                    + db
                                    + "_results_for_"
                                    + json_dirALL[0].split("_")[-1]
                                    + "_"
                                    + "formula.csv"
                                )
                                msp.loc[
                                    mz, "sirius_result_dir"
                                ] = result_sirius_name

                                ALL_formula_csv.to_csv(
                                    sub_dir
                                    + db
                                    + "_results_for_"
                                    + json_dirALL[0].split("_")[-1]
                                    + "_"
                                    + "formula.csv"
                                )

                            else:
                                print("no file for formula")
                        else:
                            print("no file for structure or formula")
            msp.to_csv(msp_csv)